In [1]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from tqdm import tqdm

from main import StateAnalysis, ChiralActiveMatter, ChiralActiveMatterNonreciprocalReact   # 替换为你的模块路径

In [ ]:
# 数据文件夹路径
data_folder = r"data"

# 定义筛选条件
param_sets = [
    {'Lambdas': [0.01], 'D0Means': [0.1], 'D0Stds': [0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2]},
    # {'Lambdas': [0.01], 'D0Means': [0.25], 'D0Stds': [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]},
    # {'Lambdas': [0.01], 'D0Means': [0.5], 'D0Stds': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]},
    # {'Lambdas': [0.01], 'D0Means': [0.75], 'D0Stds': [0, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05, 1.2, 1.35, 1.5]},
    # {'Lambdas': [0.01], 'D0Means': [1], 'D0Stds': [0, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]},
    # {'Lambdas': [0.025], 'D0Means': [0.1], 'D0Stds': [0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2]},
    # {'Lambdas': [0.025], 'D0Means': [0.25], 'D0Stds': [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]},
    # {'Lambdas': [0.025], 'D0Means': [0.5], 'D0Stds': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]},
    # {'Lambdas': [0.025], 'D0Means': [0.75], 'D0Stds': [0, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05, 1.2, 1.35, 1.5]},
    # {'Lambdas': [0.025], 'D0Means': [1], 'D0Stds': [0, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]},
]

In [4]:
d0Distribution: str = "uniform"
omegaDistribution: str = "uniform"
chiralNum: int = 1
randomSeed: int = 10

models = (
    ChiralActiveMatterNonreciprocalReact(
        strengthLambda=strengthLambda,
        distanceD0Mean=distanceD0Mean,
        distanceD0Std=distanceD0Std,
        chiralNum=chiralNum,
        agentsNum=1000,
        dt=0.01,
        tqdm=False,
        savePath=data_folder,  # 确保 savePath 指向数据文件夹
        shotsnaps=5,
        d0Distribution=d0Distribution,
        omegaDistribution=omegaDistribution,
        randomSeed=randomSeed,
        overWrite=False
    )
    for strengthLambda in param_sets[0]['Lambdas']
    for distanceD0Mean in param_sets[0]['D0Means']
    for distanceD0Std in param_sets[0]['D0Stds']
)

In [ ]:
# calc R
def calc_rail_mean_R(model: ChiralActiveMatterNonreciprocalReact):
    sa = StateAnalysis(model)
    totalPhaseTheta = sa.totalPhaseTheta

    RPool = []
    lookIdxs = np.arange(-601, 0, 3)
    for idx in lookIdxs:
        lastPhaseTheta = totalPhaseTheta[idx]
        RPool.append(StateAnalysis._clac_phase_sync_op(lastPhaseTheta))

    return np.mean(RPool)

Rs = dict()

for model in tqdm(models):
    R = calc_rail_mean_R(model)
    Rs[model] = R

In [ ]:
# calc Rc
def calc_rail_mean_Rc(model: ChiralActiveMatterNonreciprocalReact):
    sa = StateAnalysis(model)
    totalPositionX = sa.totalPositionX
    totalPhaseTheta = sa.totalPhaseTheta
    totalPointTheta = sa.totalPointTheta

    RcPool = []
    lookIdxs = np.arange(-601, 0, 5)
    for idx in lookIdxs:
        lastPositionX = totalPositionX[idx]
        lastPhaseTheta = totalPhaseTheta[idx]
        lastPointTheta = totalPointTheta[idx]

        centers = StateAnalysis._calc_centers(lastPositionX, lastPhaseTheta, lastPointTheta, model.speedV, model.dt)
        centers = np.mod(centers, 10)
        classes = StateAnalysis._calc_classes(
            centers, 0.3, 
            StateAnalysis._adj_distance(centers, centers[:, np.newaxis], 10, 5)
        )
        counts = 0
        sumR = 0
        for classOcsis in classes:
            if len(classOcsis) <= 5:
                continue
            sumR += StateAnalysis._clac_phase_sync_op(lastPhaseTheta[classOcsis])
            counts += 1

        RcPool.append(sumR / counts)

    return np.mean(RcPool)

Rcs = dict()

for model in models:
    Rc = calc_rail_mean_Rc(model)
    Rcs[model.distanceD0Std] = Rc

In [ ]:
# calc Delta Omega
def calc_delta_omega(model):
    sa = StateAnalysis(model, lookIndex=-30)
    totalPointTheta = sa.totalPointTheta
    centers = sa.centers
    classes = StateAnalysis._calc_classes(
        centers, 0.35, 
        StateAnalysis._adj_distance(centers, centers[:, np.newaxis], 10, 5)
    )
    counts = 0
    sumR = 0

    for classOcsis in classes:
        if len(classOcsis) < 5:
            continue
        meanPointTheta = totalPointTheta[-30:, classOcsis].mean(axis=0) / model.dt
        sumR += ((meanPointTheta - meanPointTheta[:, np.newaxis])**2).sum() / len(classOcsis) ** 2
        counts += 1

    return sumR / counts

delta_omegas = dict()

for model in models:
    delta_omega = calc_delta_omega(model)
    delta_omegas[model.distanceD0Std] = delta_omega

In [ ]:
# calc Nr
def clac_Nr(model: ChiralActiveMatterNonreciprocalReact):
    sa = StateAnalysis(model)
    lastPositionX, _, _ = sa.get_state(index=-1)
    centers = lastPositionX
    classes = StateAnalysis._calc_classes(
        centers, 0.4, 
        StateAnalysis._adj_distance(centers, centers[:, np.newaxis], 10, 5)
    )
    counts = 0
    ratios = 0
    for classOcsis in classes:
        if len(classOcsis) < 2:
            continue
        ratios += len(classOcsis) / model.agentsNum
        counts += 1
    return ratios / counts

Nrs = dict()

for model in models:
    Nr = clac_Nr(model)
    Nrs[model.distanceD0Std] = Nr